Соединяем датасет от 24.02.21 и датасет от 05.03.2021

In [1]:
import pandas as pd
train = pd.read_csv('train_05_03_2021.csv')
# Тестовый датасет
test = pd.read_csv('test.csv')
moscow_24_02 = pd.read_csv('moscow_24_02_2021.csv')

In [60]:
def find_data_type_discrepancy(train, test):
    """
    Функция ищет различия в типах данных 
    между датасетами и выводит информацию на экран.
    Ничего не возвращает
    :param train: DataFrame object with training data
    :param test: DataFrame object with testing data
    """
    
    train_dtypes_df = train.dtypes.reset_index()
    train_dtypes_df.columns = 'col_name type_train'.split(' ')


    test_dtypes_df = test.dtypes.reset_index()
    test_dtypes_df.columns = 'col_name type_test'.split(' ')


    merged_df = train_dtypes_df.merge(test_dtypes_df, how='outer', left_on='col_name', right_on='col_name')
    merged_df

    print(f"{'Расхождение типов данных:'}")
    print(f"{'Признак'.ljust(21)} | {'train'.ljust(15)} | {'test'.ljust(15)}")
    print('-'*48)

    for index, row in merged_df.iterrows():
        if str(row['type_train']) == 'nan' or str(row['type_test']) == 'nan':
            print(f"{str(row['col_name']).ljust(21)} | {str(row['type_train']).ljust(15)} | {str(row['type_test']).ljust(15)}")

        elif row['type_train'] != row['type_test']:
            print(f"{str(row['col_name']).ljust(21)} | {str(row['type_train']).ljust(15)} | {str(row['type_test']).ljust(15)}")
    
    

In [61]:
train['sell_id'].duplicated(keep='first').sum()

0

In [62]:
# Посмотрим на различия в типах данных между трейном и тестом
find_data_type_discrepancy(train,moscow_24_02)

Расхождение типов данных:
Признак               | train           | test           
------------------------------------------------
price                 | int64           | float64        


In [63]:
moscow_24_02['price'] = moscow_24_02['price'].astype('int64')

In [64]:
train = pd.concat([train,moscow_24_02], ignore_index=True)

In [65]:
train['sell_id'].duplicated(keep='first').sum()

22360

In [66]:
train.drop_duplicates(subset=['sell_id'], keep='last', inplace=True)

In [67]:
train.shape

(75028, 35)

In [68]:
# Посмотрим на различия в типах данных между трейном и тестом
find_data_type_discrepancy(train,test)

Расхождение типов данных:
Признак               | train           | test           
------------------------------------------------
engineDisplacement    | int64           | object         
enginePower           | int64           | object         
parsing_unixtime      | float64         | int64          
section               | object          | nan            
url_saleid            | object          | nan            
Владельцы             | float64         | object         
price                 | int64           | nan            
auto_class            | object          | nan            
price_segment         | object          | nan            
seller_type           | object          | nan            
car_url               | nan             | object         
complectation_dict    | nan             | object         
Владение              | nan             | object         


In [69]:
# Функции для приведения датасетов к единому формату

# тест -  удалим LTR и так же приведем в 1600, 1800, etc.
def get_correct_engineDisplacement_test(x):
    if x == ' LTR':  # у электроавто объем равен 0 
        return 0
    
    x = float(x.split(' LTR')[0])
    
    return int(x * 1000)

In [70]:
# engineDisplacement
# трейн - переводим к формату 1600 и тд

train.engineDisplacement = train.engineDisplacement.apply(lambda x: (x // 100) * 100)

# тест -  удалим LTR и так же приведем в 1600 и тд
test.engineDisplacement = test.engineDisplacement.apply(get_correct_engineDisplacement_test)

# enginePower
# приведем признак в тесте в типу данных int64 и уберем ' N12'
test.enginePower = test.enginePower.apply(lambda x: int(x.split(' N12')[0]))

# Владельцы
# приведем к типу float64 для совместимости с трейном
test.Владельцы = test.Владельцы.apply(lambda x: x[0]).astype('float64')

# Дата получения данных. 
# приведем к тому же формату, что и в тесте
train.parsing_unixtime = train.parsing_unixtime.astype('int64')

In [71]:
# Ограничим трейн только теми моделями и регионом, которые в тесте
test_brands = test.brand.value_counts().index.to_list()

In [72]:
#Оставим только те модели, которые есть в тесте
train = train[train.brand.isin(test_brands)]

In [73]:
train.shape

(64331, 35)

In [74]:
test.shape

(34686, 32)

In [75]:
# Посмотрим на различия в типах данных между трейном и тестом
find_data_type_discrepancy(train,test)

Расхождение типов данных:
Признак               | train           | test           
------------------------------------------------
section               | object          | nan            
url_saleid            | object          | nan            
price                 | int64           | nan            
auto_class            | object          | nan            
price_segment         | object          | nan            
seller_type           | object          | nan            
car_url               | nan             | object         
complectation_dict    | nan             | object         
Владение              | nan             | object         


In [76]:
# формируем общий датасет, размечаем тест и трейн

train['train'] = 1
test['train'] = 0

df = pd.concat([train, test], ignore_index=True)
df['sell_id'].duplicated(keep='last').sum()

3608

In [77]:
df.drop_duplicates(subset=['sell_id'], keep='last', inplace=True)

In [78]:
df[df.train==0].shape

(34686, 39)

In [79]:
df[df.train==0].price.isna().sum()

34686

In [80]:
df.to_csv("train_test_final.csv", index=False, encoding='utf-8')